In [ ]:
#embedding text
import numpy as np
import csv
import sys
import pickle

print("Start loading Word Vector Dictionary")
wordVec={}
with open('glove.txt', encoding="utf8") as glove:
    count=0
    for line in glove:
        temp=line.split()
        l=len(temp)
        wordVec[' '.join(temp[0:l-50])]=list(map(np.float,temp[l-50:l]))
        count=count+1
        print(count)
print("Finish loading Word Vector Dictionary")
print("Start loading training stances\n")
csv.field_size_limit(999999999)
with open('train_stances.csv', encoding="utf8") as csvfile_stance:
    stanceReader=csv.reader(csvfile_stance)
    stances=[]
    for row in stanceReader:
        temp=[]
        for c,c_ in zip(row[0],row[0][1:]):
            if c.isalnum() or c.isspace():
                temp.append(c.lower())
            else:
                if not c_.isspace():
                    temp.append(" ")
        stances.append([''.join(temp),row[1],row[2]])
print("Finish loading training stances\n")
print("Start loading training bodies\n")
with open('train_bodies.csv', encoding="utf8") as csvfile_body:
    bodyReader=csv.reader(csvfile_body)
    bodies={}
    for row in bodyReader:
        temp=[]
        for c,c_ in zip(row[1],row[1][1:]):
            if c.isalnum() or c.isspace():
                temp.append(c.lower())
            else:
                if not c_.isspace():
                    temp.append(" ")
        bodies[row[0]]=''.join(temp)
print("Finish loading training bodies")
print("Start merging training stances and training bodies\n")
raw_training_set=[]
for set in stances:
    raw_training_set.append([set[0],bodies[set[1]],set[2]])
    #print(bodies[key])

with open("data.p","wb") as embedding:
    pickle.dump(raw_training_set,embedding)
print("Finish loading training stanes and training bodies\n")
stanceReader=None
bodyReader=None
stances=None
bodies=None
print("Start embedding\n")

with open('data_related.p', "wb") as embedded_data_related:
    embedded_data_stances=open('data_stances.p','wb')
    data_related=[]
    data_stances=[] 
    for sample in raw_training_set:
        title=sample[0].split()
        body=sample[1].split()
        label=sample[2]
        titleVec=[]
        bodyVec=[]
        for i in range(len(title)):
            #print(title[i] in wordVec)
            if title[i] in wordVec:
                word=wordVec[title[i]]
                #print(wordVec["the"])
                titleVec.append(word)
        for i in range(len(body)):
            if body[i] in wordVec:
                word=wordVec[body[i]]
                bodyVec.append(word)
        if label =="unrelated":
            data_related.append([titleVec,bodyVec,[1,0]])
        else:
            data_related.append([titleVec,bodyVec,[0,1]])
            if label == "discuss":
                label=[1,0,0]
            if label == "agree":
                label=[0,1,0]
            if label == "disagree":
                label=[0,0,1]
            data_stances.append([titleVec,bodyVec,label])
        #data.append([titleVec,bodyVec,label])
    pickle.dump(np.array(data_related),embedded_data_related)
    pickle.dump(np.array(data_stances),embedded_data_stances)



In [ ]:
#related-level1
# -*- coding: utf-8 -*-
"""
Created on Fri Mar 30 22:15:39 2018

@author: HP
"""

import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import pickle

class dataWrapper:
    def __init__(self,data):
        self.size=len(data)
        self.x_title=[]
        self.x_body=[]
        self.y=[]
        self.seqlen_title=[]
        self.seqlen_body=[]
        self.current_batch=0
        for sample in data:
            self.x_title.append(sample[0])
            self.x_body.append(sample[1])
            self.y.append(sample[2])
            self.seqlen_title.append(len(sample[0]))
            self.seqlen_body.append(len(sample[1]))
        max_seqlen=max(self.seqlen_body+self.seqlen_title)
        #padding the samples with zero vectors
        for i in range(len(self.x_title)):
            self.x_title[i]+=[[0]*50]*(max_seqlen-len(self.x_title[i]))
            self.x_body[i]+=[[0]*50]*(max_seqlen-len(self.x_body[i]))
    # return the next batch of the data from the data set.
    def next(self,batch_size):
        if self.current_batch+batch_size<self.size:
            self.current_batch+=batch_size
            return self.x_title[self.current_batch:self.current_batch+batch_size],self.x_body[self.current_batch:self.current_batch+batch_size],self.y[self.current_batch:self.current_batch+batch_size],self.seqlen_title[self.current_batch:self.current_batch+batch_size],self.seqlen_body[self.current_batch:self.current_batch+batch_size]
        else:
            temp=self.current_batch
            self.current_batch=self.current_batch+batch_size-self.size
            batch_x_title=self.x_title[temp:]+self.x_title[:self.current_batch]
            batch_x_body=self.x_body[temp:]+self.x_body[:self.current_batch]
            batch_y=self.y[temp:]+self.y[:self.current_batch]
            batch_seqlen_title=self.seqlen_title[temp:]+self.seqlen_title[:self.current_batch]
            batch_seqlen_body=self.seqlen_body[temp:]+self.seqlen_body[:self.current_batch]
            return batch_x_title,batch_x_body,batch_y,batch_seqlen_title,batch_seqlen_body
    # return the length of the longest sequence
    def max_seqlen(self):
        return max(self.seqlen_body+self.seqlen_title)

# loading the data
data=pickle.load(open("data_stances.p","rb"))
size=len(data)
trainset=dataWrapper(data[size//3:])
testset=dataWrapper(data[:size//3])

#network parameters
learning_rate=0.001
training_iters=100000
batch_size=128
display_step=10

seq_max_len=max(trainset.max_seqlen(),testset.max_seqlen())
n_input=50
n_hidden=60
n_classes=3


x_title=tf.placeholder("float",[None,seq_max_len,n_input])
x_body=tf.placeholder("float",[None,seq_max_len,n_input])
y=tf.placeholder("float",[None,n_classes])
seqlen_title=tf.placeholder(tf.int32,[None])
seqlen_body=tf.placeholder(tf.int32,[None])


weights={'out':tf.Variable(tf.random_normal([n_hidden,n_classes]))}
biases={'out':tf.Variable(tf.random_normal([1,n_classes]))}

def dynamicRNN(x_title,x_body,seqlen_title,seqlen_body,weights,biases):

    gru_cell=rnn.GRUCell(num_units=n_hidden)
    #rnn.BasicRNNCell()
    print("testing_1")
    outputs_title,states_title=tf.nn.dynamic_rnn(cell=gru_cell,inputs=x_title,sequence_length=seqlen_title,dtype=tf.float32)

    with tf.variable_scope('scope1',reuse=None):    
        print("testing_2")
        outputs_body,states_body=tf.nn.dynamic_rnn(cell=gru_cell,inputs=x_body,sequence_length=seqlen_body,dtype=tf.float32)
        print("testing_3")
        temp1=tf.stack([tf.range(tf.shape(seqlen_title)[0]),seqlen_title-1],axis=1)
        temp2=tf.stack([tf.range(tf.shape(seqlen_body)[0]),seqlen_body-1],axis=1)
        return tf.matmul(tf.multiply(tf.gather_nd(outputs_title,temp1),tf.gather_nd(outputs_body,temp2)),weights['out'])+biases['out']

pred=dynamicRNN(x_title,x_body,seqlen_title,seqlen_body,weights,biases)
cost =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    step=1

    while step*batch_size<training_iters:
        print("step:",step)

        batch_x_title,batch_x_body,batch_y,batch_seqlen_title,batch_seqlen_body=trainset.next(batch_size)
        sess.run(optimizer,feed_dict={x_title:batch_x_title,x_body:batch_x_body,y:batch_y,seqlen_title:batch_seqlen_title,seqlen_body:batch_seqlen_body})
        if step%display_step==0:
            acc=sess.run(accuracy,feed_dict={x_title:batch_x_title,x_body:batch_x_body,y:batch_y,seqlen_title:batch_seqlen_title,seqlen_body:batch_seqlen_body})
            loss=sess.run(cost,feed_dict={x_title:batch_x_title,x_body:batch_x_body,y:batch_y,seqlen_title:batch_seqlen_title,seqlen_body:batch_seqlen_body})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step+=1
    print("Optimization Finished!")

test_x_title=testset.x_title
test_x_body=testset.x_body
test_y=testset.y
test_seqlen_title=testset.seqlen_title
test_seqlen_body=testset.seqlen_body

print("Test Accuracy:",sess.run(accuracy,feed_dict={x_title:test_x_title,x_body:test_x_body,y:test_y,seqlen_title:test_seqlen_title,seqlen_body:test_seqlen_body}))
   

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import pickle
import sys
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
import os; os.environ['CUDA_VISIBLE_DEVICES'] = sys.argv[1]
class dataWrapper:
    def __init__(self,data):
        self.size=len(data)
        self.x_title=[]
        self.x_body=[]
        self.y=[]
        self.seqlen_title=[]
        self.seqlen_body=[]
        self.current_batch=0
        for sample in data:
            self.x_title.append(sample[0])
            self.x_body.append(sample[1])
            self.y.append(sample[2])
            self.seqlen_title.append(len(sample[0]))
            self.seqlen_body.append(len(sample[1]))
        max_seqlen=6000
        #padding the samples with zero vectors
        for i in range(len(self.x_title)):
            self.x_title[i]+=[[0]*50]*(max_seqlen-len(self.x_title[i]))
            self.x_body[i]+=[[0]*50]*(max_seqlen-len(self.x_body[i]))
    # return the next batch of the data from the data set.
    def next(self,batch_size):
        if self.current_batch+batch_size<self.size:
            self.current_batch+=batch_size
            return self.x_title[self.current_batch:self.current_batch+batch_size],self.x_body[self.current_batch:self.current_batch+batch_size],self.y[self.current_batch:self.current_batch+batch_size],self.seqlen_title[self.current_batch:self.current_batch+batch_size],self.seqlen_body[self.current_batch:self.current_batch+batch_size]
        else:
            temp=self.current_batch
            self.current_batch=self.current_batch+batch_size-self.size
            batch_x_title=self.x_title[temp:]+self.x_title[:self.current_batch]
            batch_x_body=self.x_body[temp:]+self.x_body[:self.current_batch]
            batch_y=self.y[temp:]+self.y[:self.current_batch]
            batch_seqlen_title=self.seqlen_title[temp:]+self.seqlen_title[:self.current_batch]
            batch_seqlen_body=self.seqlen_body[temp:]+self.seqlen_body[:self.current_batch]
            return batch_x_title,batch_x_body,batch_y,batch_seqlen_title,batch_seqlen_body
    # return the length of the longest sequence
    def max_seqlen(self):
        return max(self.seqlen_body+self.seqlen_title)

# loading the data
data=pickle.load(open("data_stances.p","rb"))
size=len(data)
trainset=np.array(data[800:])
np.random.shuffle(trainset)
trainset=dataWrapper(trainset)
testset=dataWrapper(data[:800])
data=None
#network parameters
learning_rate=0.001
#training_iters=100000
training_iters=3*trainset.size
batch_size=128
display_step=10

seq_max_len=6000
n_input=50
n_hidden=100
n_classes=3


x_title=tf.placeholder("float",[None,seq_max_len,n_input])
x_body=tf.placeholder("float",[None,seq_max_len,n_input])
y=tf.placeholder("float",[None,n_classes])
seqlen_title=tf.placeholder(tf.int32,[None])
seqlen_body=tf.placeholder(tf.int32,[None])


weights={'out':tf.Variable(tf.random_normal([n_hidden,n_classes]))}
biases={'out':tf.Variable(tf.random_normal([1,n_classes]))}

def dynamicRNN(x_title,x_body,seqlen_title,seqlen_body,weights,biases):

    lstm_cell=rnn.GRUCell(n_hidden)
    print("testing_1")
    outputs_title,states_title=tf.nn.dynamic_rnn(cell=lstm_cell,inputs=x_title,sequence_length=seqlen_title,dtype=tf.float32)

    with tf.variable_scope('scope2',reuse=None):    
        print("testing_2")
        outputs_body,states_body=tf.nn.dynamic_rnn(cell=lstm_cell,inputs=x_body,sequence_length=seqlen_body,dtype=tf.float32)
        print("testing_3")
        temp1=tf.stack([tf.range(tf.shape(seqlen_title)[0]),seqlen_title-1],axis=1)
        temp2=tf.stack([tf.range(tf.shape(seqlen_body)[0]),seqlen_body-1],axis=1)
        return tf.matmul(tf.multiply(tf.gather_nd(outputs_title,temp1),tf.gather_nd(outputs_body,temp2)),weights['out'])+biases['out']

pred=dynamicRNN(x_title,x_body,seqlen_title,seqlen_body,weights,biases)
cost =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init=tf.global_variables_initializer()

saver=tf.train.Saver()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.device('/device:GPU:0'):
    sess=tf.Session(config=config)
    sess.run(init)
    step=1
    saver.restore(sess, "/uac/y15/kcli5/FNC-1/tmp2/model_stance_1.0-290")
    print("Model restored.")
    while step*batch_size<training_iters:
        print("step:",step)

        batch_x_title,batch_x_body,batch_y,batch_seqlen_title,batch_seqlen_body=trainset.next(batch_size)
        sess.run(optimizer,feed_dict={x_title:batch_x_title,x_body:batch_x_body,y:batch_y,seqlen_title:batch_seqlen_title,seqlen_body:batch_seqlen_body})
        if step%display_step==0:
            acc=sess.run(accuracy,feed_dict={x_title:batch_x_title,x_body:batch_x_body,y:batch_y,seqlen_title:batch_seqlen_title,seqlen_body:batch_seqlen_body})
            loss=sess.run(cost,feed_dict={x_title:batch_x_title,x_body:batch_x_body,y:batch_y,seqlen_title:batch_seqlen_title,seqlen_body:batch_seqlen_body})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
            save_path = saver.save(sess, "/uac/y15/kcli5/FNC-1/tmp2/model_stance_1.0",global_step=step+540)
            print("Model saved in file: %s" % save_path)
        step+=1
    print("Optimization Finished!")
    test_x_title=testset.x_title
    test_x_body=testset.x_body
    test_y=testset.y
    test_seqlen_title=testset.seqlen_title
    test_seqlen_body=testset.seqlen_body
    print("Test Accuracy:",sess.run(accuracy,feed_dict={x_title:test_x_title,x_body:test_x_body,y:test_y,seqlen_title:test_seqlen_title,seqlen_body:test_seqlen_body}))
   

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import pickle

class dataWrapper:
    def __init__(self,data):
        self.size=len(data)
        self.x_title=[]
        self.x_body=[]
        self.y=[]
        self.seqlen_title=[]
        self.seqlen_body=[]
        self.current_batch=0
        for sample in data:
            self.x_title.append(sample[0])
            self.x_body.append(sample[1])
            self.y.append(sample[2])
            self.seqlen_title.append(len(sample[0]))
            self.seqlen_body.append(len(sample[1]))
        max_seqlen=max(self.seqlen_body+self.seqlen_title)
        #padding the samples with zero vectors
        for i in range(len(self.x_title)):
            self.x_title[i]+=[[0]*50]*(max_seqlen-len(self.x_title[i]))
            self.x_body[i]+=[[0]*50]*(max_seqlen-len(self.x_body[i]))
    # return the next batch of the data from the data set.
    def next(self,batch_size):
        if self.current_batch+batch_size<self.size:
            self.current_batch+=batch_size
            return self.x_title[self.current_batch:self.current_batch+batch_size],self.x_body[self.current_batch:self.current_batch+batch_size],self.y[self.current_batch:self.current_batch+batch_size],self.seqlen_title[self.current_batch:self.current_batch+batch_size],self.seqlen_body[self.current_batch:self.current_batch+batch_size]
        else:
            temp=self.current_batch
            self.current_batch=self.current_batch+batch_size-self.size
            batch_x_title=self.x_title[temp:]+self.x_title[:self.current_batch]
            batch_x_body=self.x_body[temp:]+self.x_body[:self.current_batch]
            batch_y=self.y[temp:]+self.y[:self.current_batch]
            batch_seqlen_title=self.seqlen_title[temp:]+self.seqlen_title[:self.current_batch]
            batch_seqlen_body=self.seqlen_body[temp:]+self.seqlen_body[:self.current_batch]
            return batch_x_title,batch_x_body,batch_y,batch_seqlen_title,batch_seqlen_body
    # return the length of the longest sequence
    def max_seqlen(self):
        return max(self.seqlen_body+self.seqlen_title)

# loading the data
data=pickle.load(open("data_stances.p","rb"))
size=len(data)
trainset=dataWrapper(data[size//3:])
testset=dataWrapper(data[:size//3])

#network parameters
learning_rate=0.001
training_iters=100000
batch_size=128
display_step=10

seq_max_len=max(trainset.max_seqlen(),testset.max_seqlen())
n_input=50
n_hidden=60
n_classes=3


x_title=tf.placeholder("float",[None,seq_max_len,n_input])
x_body=tf.placeholder("float",[None,seq_max_len,n_input])
y=tf.placeholder("float",[None,n_classes])
seqlen_title=tf.placeholder(tf.int32,[None])
seqlen_body=tf.placeholder(tf.int32,[None])


weights={'out':tf.Variable(tf.random_normal([n_hidden,n_classes]))}
biases={'out':tf.Variable(tf.random_normal([1,n_classes]))}

def dynamicRNN(x_title,x_body,seqlen_title,seqlen_body,weights,biases):

    gru_cell=rnn.GRUCell(n_hidden)
    print("testing_1")
    outputs_title,states_title=tf.nn.dynamic_rnn(cell=gru_cell,inputs=x_title,sequence_length=seqlen_title,dtype=tf.float32)

    with tf.variable_scope('scope1',reuse=None):    
        print("testing_2")
        outputs_body,states_body=tf.nn.dynamic_rnn(cell=gru_cell,inputs=x_body,sequence_length=seqlen_body,dtype=tf.float32)
        print("testing_3")
        temp1=tf.stack([tf.range(tf.shape(seqlen_title)[0]),seqlen_title-1],axis=1)
        temp2=tf.stack([tf.range(tf.shape(seqlen_body)[0]),seqlen_body-1],axis=1)
        return tf.matmul(tf.multiply(tf.gather_nd(outputs_title,temp1),tf.gather_nd(outputs_body,temp2)),weights['out'])+biases['out']

pred=dynamicRNN(x_title,x_body,seqlen_title,seqlen_body,weights,biases)
cost =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    step=1

    while step*batch_size<training_iters:
        print("step:",step)

        batch_x_title,batch_x_body,batch_y,batch_seqlen_title,batch_seqlen_body=trainset.next(batch_size)
        sess.run(optimizer,feed_dict={x_title:batch_x_title,x_body:batch_x_body,y:batch_y,seqlen_title:batch_seqlen_title,seqlen_body:batch_seqlen_body})
        if step%display_step==0:
            acc=sess.run(accuracy,feed_dict={x_title:batch_x_title,x_body:batch_x_body,y:batch_y,seqlen_title:batch_seqlen_title,seqlen_body:batch_seqlen_body})
            loss=sess.run(cost,feed_dict={x_title:batch_x_title,x_body:batch_x_body,y:batch_y,seqlen_title:batch_seqlen_title,seqlen_body:batch_seqlen_body})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step+=1
    print("Optimization Finished!")

test_x_title=testset.x_title
test_x_body=testset.x_body
test_y=testset.y
test_seqlen_title=testset.seqlen_title
test_seqlen_body=testset.seqlen_body

print("Test Accuracy:",sess.run(accuracy,feed_dict={x_title:test_x_title,x_body:test_x_body,y:test_y,seqlen_title:test_seqlen_title,seqlen_body:test_seqlen_body}))
   